In [1]:
import rdflib
import urllib
import pprint
import itertools
import os
import sys
from tqdm import tqdm

sys.path.append(os.path.dirname(os.getcwd()))
from data import WN18

In [2]:
def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples

def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    triples_all = triples_tr + triples_va + triples_te
    return triples_all, triples_tr, triples_va, triples_te

def get_resources(triples):
    return set([r for triple in triples for r in triple])

def to_uri(suffix, uri_prefix):
    return uri_prefix + urllib.parse.quote(suffix)

def glance_dict(d, n=5):
    return dict(itertools.islice(d.items(), n))

def build_rdf_graph(triples, uri_prefix='http://www.example.org/'):
    graph = rdflib.Graph()
    resources = get_resources(triples)
    resource2uri = {r: to_uri(r, uri_prefix) for r in resources}
    uri2resource = {uri: r for r, uri in resource2uri.items()}
    for (s, p, o) in tqdm(triples, total=len(triples), ncols=70):
        s_uri = rdflib.URIRef(resource2uri[s])
        p_uri = rdflib.URIRef(resource2uri[p])
        o_uri = rdflib.URIRef(resource2uri[o])
        graph.add((s_uri, p_uri, o_uri))
    return graph, uri2resource

In [3]:
triples_all, triples_tr, triples_va, triples_te = load_triple()
rdf_graph, uri2resource = build_rdf_graph(triples_tr)
query = (
    """
    SELECT DISTINCT ?s ?p ?o WHERE {
        ?s ?p ?o .
    }
    """
)
for row in rdf_graph.query(query):
    s = uri2resource[row.s.toPython()]
    p = uri2resource[row.p.toPython()]
    o = uri2resource[row.o.toPython()]
    print(s, p, o)
    break

Files Already Downloaded


100%|██████████████████████| 141442/141442 [00:05<00:00, 27756.85it/s]


09316454 _instance_hyponym 08908954
